In [2]:
import json
import networkx as nx
from networkx.algorithms.simple_paths import shortest_simple_paths
from collections import defaultdict

with open("cable-and-landing-points.json", "r") as f:
    cable_and_landing_points = json.load(f)

allowed_countries = {
    "India", "Australia", "Sri Lanka", "Myanmar", "Thailand", "Cambodia",
    "Vietnam", "Malaysia", "Indonesia", "Singapore", "Papua New Guinea",
    "Brunei", "Timor-Leste", "Philippines", "Solomon Islands"
}

def normalize_node(lp):
    return f"{lp['city'].strip()}, {lp['country'].strip()}"

def get_coords(lp):
    return tuple(lp["coordinates"])

G = nx.Graph()
node_coords = {}
india_nodes = set()
australia_nodes = set()
allowed_nodes = set()
cable_to_nodes = defaultdict(set)

for cable in cable_and_landing_points:
    cable_name = cable["cable_name"]
    lps = cable["landing_points"]

    for i in range(len(lps)):
        lp_i = lps[i]
        node_i = normalize_node(lp_i)
        coord_i = get_coords(lp_i)
        country_i = lp_i["country"]
        node_coords[node_i] = coord_i
        cable_to_nodes[cable_name].add(node_i)

        if country_i == "India":
            india_nodes.add(node_i)
        elif country_i == "Australia":
            australia_nodes.add(node_i)

        if country_i in allowed_countries:
            allowed_nodes.add(node_i)

        for j in range(i + 1, len(lps)):
            lp_j = lps[j]
            node_j = normalize_node(lp_j)
            coord_j = get_coords(lp_j)
            country_j = lp_j["country"]
            node_coords[node_j] = coord_j
            cable_to_nodes[cable_name].add(node_j)

            if country_j in allowed_countries:
                allowed_nodes.add(node_j)

            G.add_edge(node_i, node_j, cable=cable_name)

single_cable_paths = []
for cable, nodes in cable_to_nodes.items():
    ind_in_nodes = india_nodes & nodes
    aus_in_nodes = australia_nodes & nodes

    if not ind_in_nodes or not aus_in_nodes:
        continue

    subgraph = G.subgraph(nodes)
    for source in ind_in_nodes:
        for target in aus_in_nodes:
            try:
                path = nx.shortest_path(subgraph, source, target)
                single_cable_paths.append({
                    "path": path,
                    "cables": [cable]
                })
            except nx.NetworkXNoPath:
                continue

subG = G.subgraph(allowed_nodes)
multi_cable_paths = []
MAX_PATHS_PER_PAIR = 10

for source in india_nodes:
    for target in australia_nodes:
        try:
            path_gen = shortest_simple_paths(subG, source, target)
            count = 0
            for path in path_gen:
                if count >= MAX_PATHS_PER_PAIR:
                    break

                cables = []
                valid = True
                for i in range(len(path) - 1):
                    edge_data = subG.get_edge_data(path[i], path[i+1])
                    if edge_data and "cable" in edge_data:
                        cables.append(edge_data["cable"])
                    else:
                        valid = False
                        break

                if valid:
                    multi_cable_paths.append({
                        "path": path,
                        "cables": cables
                    })
                    count += 1
        except nx.NetworkXNoPath:
            continue

#######################
MAX_CABLES_ALLOWED = 2 
####################### 

filtered_paths = []
for item in single_cable_paths + multi_cable_paths:
    cable_count = len(set(item["cables"]))
    if cable_count <= MAX_CABLES_ALLOWED:
        filtered_paths.append(item)

all_valid_paths = sorted(filtered_paths, key=lambda x: len(set(x["cables"])))


geojson_features = []
for item in all_valid_paths:
    coordinates = [node_coords[n] for n in item["path"]]
    feature = {
        "type": "Feature",
        "properties": {
            "path": item["path"],
            "cables": item["cables"],
            "cable_count": len(set(item["cables"]))
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        }
    }
    geojson_features.append(feature)

geojson_output = {
    "type": "FeatureCollection",
    "features": geojson_features
}

# with open("codes/india_australia_paths.geojson", "w") as f:
#     json.dump(geojson_output, f, indent=2)

# with open("codes/india_australia_paths_2_cables.geojson", "w") as f:
#     json.dump(geojson_output, f, indent=2)

print(f"Saved {len(geojson_features)} paths using ≤ {MAX_CABLES_ALLOWED} cables.")

for item in all_valid_paths:
    print(" → ".join(item["path"]))
    print("  Cables:", " → ".join(item["cables"]))
    print()

Saved 63 paths using ≤ 2 cables.
Chennai, India → Tuas, Singapore → Perth, Australia
  Cables: SeaMeWe-4 → INDIGO-West

Chennai, India → Jakarta, Indonesia → Perth, Australia
  Cables: ICE IV → INDIGO-West

Chennai, India → Ancol, Indonesia → Perth, Australia
  Cables: ICE IV → INDIGO-West

Chennai, India → Jakarta, Indonesia → Darwin, Australia
  Cables: ICE IV → Hawaiki Nui 1

Chennai, India → Singapore, Singapore → Darwin, Australia
  Cables: ICE IV → Hawaiki Nui 1

Chennai, India → Ancol, Indonesia → Darwin, Australia
  Cables: ICE IV → Hawaiki Nui 1

Chennai, India → Singapore, Singapore → Batam, Indonesia → Darwin, Australia
  Cables: ICE IV → Hawaiki Nui 1 → Hawaiki Nui 1

Chennai, India → Singapore, Singapore → Makassar, Indonesia → Darwin, Australia
  Cables: ICE IV → Asia Connect Cable-1 (ACC-1) → Asia Connect Cable-1 (ACC-1)

Chennai, India → Jakarta, Indonesia → Alexandria, Australia
  Cables: ICE IV → Hawaiki Nui 1

Chennai, India → Singapore, Singapore → Alexandria, Austr